In [1]:
import numpy as np
import torch

from tqdm.auto import tqdm
from tqdm.notebook import tqdm_notebook, trange

import itertools
import os
from dotenv import find_dotenv, load_dotenv
from pathlib import Path

load_dotenv(find_dotenv(), verbose=True)

True

# prepare data

In [2]:
from src.utils.ensemble import optimize_val_correlation
from src.config.config import combine_cfgs, get_cfg_defaults
from src.data.datamodule import MyDataModule

In [3]:
# prepare validation data
cfg = combine_cfgs('../src/config/experiments/algonauts2021_i3d_flow.yml')
dm = MyDataModule(cfg)
dm.prepare_data()
dm.setup()

val_indices = dm.val_dataset.indices

cache_path = Path('/data_smr/huze/.cache/val_fmris.pt')

if cache_path.exists():
    val_fmris = torch.load(cache_path)
else:
    val_fmris = [dm.dataset_train_val.__getitem__(i)[1] for i in tqdm(val_indices)]
    val_fmris = np.stack(val_fmris, 0)
    val_fmris = torch.tensor(val_fmris)
    torch.save(val_fmris, cache_path)

In [4]:
# load from notebook 900. (cross-notebook-ref)

In [5]:
import pandas as pd

In [6]:
submission_df = torch.load('./tmp/submission_df.pt')

In [7]:
submission_df

,model_sch,he_sch,roi_sch,cross_roi_sch,i_submission,path
0,multi_layer&i3d_rgb,H3,ROIxSMC,croi_ensemble,000,/data/huze/ray_results/algonauts2021/ensemble_...
1,single_layer&i3d_rgb+i3d_flow,H3,ROIxSC,croi_ensemble,001,/data/huze/ray_results/algonauts2021/ensemble_...
2,single_layer&i3d_rgb+i3d_flow,H3,ROI,,002,/data/huze/ray_results/algonauts2021/ensemble_...
3,single_layer&i3d_rgb+i3d_flow,H3,WB,,003,/data/huze/ray_results/algonauts2021/ensemble_...
4,single_layer&i3d_flow,H3,ROIxSMC,croi_ensemble,004,/data/huze/ray_results/algonauts2021/ensemble_...
5,single_layer&i3d_rgb+i3d_flow,H1,ROIxSMC,croi_ensemble,005,/data/huze/ray_results/algonauts2021/ensemble_...
6,single_layer&i3d_rgb+i3d_flow,H3,ROIxLC,croi_ensemble,006,/data/huze/ray_results/algonauts2021/ensemble_...
7,multi_layer&i3d_flow,H3,ROIxSMC,croi_ensemble,007,/data/huze/ray_results/algonauts2021/ensemble_...
8,single_layer&i3d_rgb+i3d_flow,H3,SMC,,008,/data/huze/ray_results/algonauts2021/ensemble_...
9,single_layer&i3d_rgb+i3d_flow,H2,ROIxSMC,croi_ensemble,009,/data/huze/ray_results/algonauts2021/ensemble_...


# validation score for each ROI and kROI

In [8]:
from src.utils.metrics import vectorized_correlation

In [9]:
rois = ['WB', 'V1', 'V2', 'V3', 'V4', 'LOC', 'EBA', 'FFA', 'STS', 'PPA', 'REST', 'SMC1', 'SMC2', 'MC2', 'MC1', 'SC3',
        'SC4', 'LC1',
        'LC2', 'LC3', 'LC4', 'LC5']

In [10]:
# validation set score

score_res_dicts = []

for i_row, row in tqdm(list(submission_df.iterrows())):
    # model_sch = row.model_sch
    # he_sch = row.he_sch
    # roi_sch = row.roi_sch

    path = row.path
    prediction = torch.load(path)
    scores = vectorized_correlation(prediction[val_indices], val_fmris)
    res_dict = {}
    for roi in rois:
        voxel_indices = torch.load(os.path.join(cfg.DATASET.VOXEL_INDEX_DIR, f'{roi}.pt'))
        score = scores[voxel_indices].mean().item()
        key = f'{roi}_val_score'
        res_dict[key] = f'{score:.4f}'
    score_res_dicts.append(res_dict)
roi_val_score_df = pd.DataFrame(score_res_dicts)

  0%|          | 0/22 [00:00<?, ?it/s]

In [11]:
all_score_df = submission_df.join(roi_val_score_df)

# whole brain submission score

In [12]:
# test set score

In [13]:
from io import StringIO

In [14]:
results = """NUM\tSCORE\tFILENAME\tSUBMISSION\tDATE\tSTATUS\t
91	0.3726401715	i=000,he_sch=H3,model_sch=multi_layer&i3d_rgb,roi_sch=ROIxSMC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:03:28	Finished		
92	0.3822759446	i=001,he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=ROIxSC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:04:23	Finished		
93	0.3658464045	i=002,he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=ROI_full_track.zip	02/28/2022 14:06:27	Finished		
94	0.3566607576	i=003,he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=WB_full_track.zip	02/28/2022 14:08:08	Finished		
95	0.3371816907	i=004,he_sch=H3,model_sch=single_layer&i3d_flow,roi_sch=ROIxSMC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:17:15	Finished		
96	0.3755210745	i=005,he_sch=H1,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=ROIxSMC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:20:31	Finished		
97	0.3765596377	i=006,he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=ROIxLC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:21:09	Finished		
98	0.3409911341	i=007,he_sch=H3,model_sch=multi_layer&i3d_flow,roi_sch=ROIxSMC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:23:47	Finished		
99	0.3777393409	i=008,he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=SMC_full_track.zip	02/28/2022 14:24:18	Finished		
100	0.3782319207	i=009,he_sch=H2,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=ROIxSMC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:25:02	Finished		
101	0.3780098575	i=010,he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=ROIxSMC,cross_roi_sch=voxel_swap_full_track.zip	02/28/2022 14:26:42	Finished		
102	0.3823800812	i=011,he_sch=H3,model_sch=single_layer+multi_layer&i3d_rgb+i3d_flow,roi_sch=ROIxSMC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:28:01	Finished		
103	0.3386854364	i=012,he_sch=H3,model_sch=single_layer+multi_layer&i3d_flow,roi_sch=ROIxSMC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:28:31	Finished		
104	0.3780980566	i=013,he_sch=H3,model_sch=single_layer+multi_layer&i3d_rgb,roi_sch=ROIxSMC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:29:19	Finished		
105	0.3823963473	i=014,he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=ROIxSMC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:30:10	Finished		
106	0.3792904924	i=015,he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=ROIxMC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:30:54	Finished		
107	0.3779275382	i=016,he_sch=H3,model_sch=single_layer&i3d_rgb,roi_sch=ROIxSMC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:31:34	Finished		
108	0.3820716002	i=017,he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=ROIxSMC,cross_roi_sch=croi_swap_full_track.zip	02/28/2022 14:32:14	Finished		
109	0.3710070445	i=018,he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=LC_full_track.zip	02/28/2022 14:33:03	Finished		
110	0.3778493967	i=019,he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=SC_full_track.zip	02/28/2022 14:33:56	Finished		
111	0.3747412699	i=020,he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=MC_full_track.zip	02/28/2022 14:34:33	Finished		
112	0.3771323259	i=021,he_sch=H3,model_sch=multi_layer&i3d_rgb+i3d_flow,roi_sch=ROIxSMC,cross_roi_sch=croi_ensemble_full_track.zip	02/28/2022 14:36:07	Finished	
"""

In [15]:
df = pd.read_csv(StringIO(results), sep="\t")

df['WB_test_score'] = df['SCORE'].apply(lambda x: f'{x:.4f}')

df = df[['WB_test_score']]

In [16]:
all_score_df = all_score_df.drop(columns=['WB_test_score'], errors='ignore')

In [17]:
all_score_df = all_score_df.join(df)

# ROI submission score

In [18]:
submission_output_dir = './submissions'

In [19]:
# test set score
import zipfile

score_res_dicts = []

for i in range(1, len(all_score_df) + 1):
    # make sure the first file is `output_file (1).zip`
    with zipfile.ZipFile(os.path.join(submission_output_dir, f'output_file ({i}).zip'), 'r') as zip_ref:
        zip_ref.extractall(submission_output_dir)
    df = pd.read_csv(os.path.join(submission_output_dir, 'scores.txt'), sep=': ', header=None, engine='python')

    res_dict = {}
    for j in range(len(df)):
        roi, score = df.iloc[j]
        if roi == 'score': continue
        key = f'{roi}_test_score'
        res_dict[key] = f'{score:.4f}'
    score_res_dicts.append(res_dict)

roi_test_score_df = pd.DataFrame(score_res_dicts)

In [20]:
all_score_df = all_score_df.drop(columns=list(roi_test_score_df.keys()), errors='ignore')

In [21]:
all_score_df = all_score_df.join(roi_test_score_df)

# preprocess table

In [22]:
print(rois)

['WB', 'V1', 'V2', 'V3', 'V4', 'LOC', 'EBA', 'FFA', 'STS', 'PPA', 'REST', 'SMC1', 'SMC2', 'MC2', 'MC1', 'SC3', 'SC4', 'LC1', 'LC2', 'LC3', 'LC4', 'LC5']


In [23]:
for roi in rois:
    val_key = f'{roi}_val_score'
    test_key = f'{roi}_test_score'
    if test_key in all_score_df.keys():
        all_score_df[roi] = all_score_df[val_key] + '/' + all_score_df[test_key]
    else:
        all_score_df[roi] = all_score_df[val_key] + '/' + '-'

In [24]:
all_score_df.keys()

Index(['model_sch', 'he_sch', 'roi_sch', 'cross_roi_sch', 'i_submission',
       'path', 'WB_val_score', 'V1_val_score', 'V2_val_score', 'V3_val_score',
       'V4_val_score', 'LOC_val_score', 'EBA_val_score', 'FFA_val_score',
       'STS_val_score', 'PPA_val_score', 'REST_val_score', 'SMC1_val_score',
       'SMC2_val_score', 'MC2_val_score', 'MC1_val_score', 'SC3_val_score',
       'SC4_val_score', 'LC1_val_score', 'LC2_val_score', 'LC3_val_score',
       'LC4_val_score', 'LC5_val_score', 'WB_test_score', 'V1_test_score',
       'V2_test_score', 'V3_test_score', 'V4_test_score', 'LOC_test_score',
       'EBA_test_score', 'FFA_test_score', 'STS_test_score', 'PPA_test_score',
       'WB', 'V1', 'V2', 'V3', 'V4', 'LOC', 'EBA', 'FFA', 'STS', 'PPA', 'REST',
       'SMC1', 'SMC2', 'MC2', 'MC1', 'SC3', 'SC4', 'LC1', 'LC2', 'LC3', 'LC4',
       'LC5'],
      dtype='object')

In [25]:
interesting_keys = ['model_sch', 'he_sch', 'roi_sch', 'cross_roi_sch'] + rois

In [26]:
full_table_df = all_score_df[interesting_keys]

# make tables

In [27]:
full_table_df

,model_sch,he_sch,roi_sch,cross_roi_sch,WB,V1,V2,V3,V4,LOC,...,SMC2,MC2,MC1,SC3,SC4,LC1,LC2,LC3,LC4,LC5
0,multi_layer&i3d_rgb,H3,ROIxSMC,croi_ensemble,0.1747/0.3726,0.3335/0.6768,0.3221/0.6889,0.3059/0.6795,0.2812/0.7183,0.3546/0.7179,...,0.0444/-,0.2873/-,0.0584/-,0.3153/-,0.2596/-,0.0876/-,0.3216/-,0.2514/-,0.4699/-,0.4191/-
1,single_layer&i3d_rgb+i3d_flow,H3,ROIxSC,croi_ensemble,0.1800/0.3823,0.3338/0.6621,0.3232/0.6818,0.3080/0.6792,0.2841/0.7226,0.3631/0.7272,...,0.0462/-,0.2984/-,0.0616/-,0.3302/-,0.2670/-,0.0919/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-
2,single_layer&i3d_rgb+i3d_flow,H3,ROI,,0.1700/0.3658,0.3333/0.6613,0.3228/0.6826,0.3061/0.6760,0.2829/0.7227,0.3582/0.7247,...,0.0417/-,0.2818/-,0.0543/-,0.3059/-,0.2581/-,0.0833/-,0.3202/-,0.2499/-,0.4723/-,0.3779/-
3,single_layer&i3d_rgb+i3d_flow,H3,WB,,0.1602/0.3567,0.2302/0.4921,0.2409/0.5266,0.2510/0.5729,0.2329/0.6239,0.3437/0.6975,...,0.0371/-,0.2670/-,0.0488/-,0.2886/-,0.2458/-,0.0766/-,0.3118/-,0.2401/-,0.4527/-,0.3264/-
4,single_layer&i3d_flow,H3,ROIxSMC,croi_ensemble,0.1565/0.3372,0.2551/0.5321,0.2619/0.5802,0.2692/0.6023,0.2530/0.6422,0.3322/0.6609,...,0.0342/-,0.2665/-,0.0476/-,0.2877/-,0.2456/-,0.0755/-,0.3022/-,0.2200/-,0.4543/-,0.3501/-
5,single_layer&i3d_rgb+i3d_flow,H1,ROIxSMC,croi_ensemble,0.1829/0.3755,0.3354/0.6587,0.3248/0.6797,0.3109/0.6704,0.2877/0.7115,0.3662/0.7220,...,0.0473/-,0.3003/-,0.0643/-,0.3310/-,0.2701/-,0.0944/-,0.3291/-,0.2672/-,0.4874/-,0.4208/-
6,single_layer&i3d_rgb+i3d_flow,H3,ROIxLC,croi_ensemble,0.1778/0.3766,0.3334/0.6615,0.3230/0.6823,0.3079/0.6794,0.2834/0.7249,0.3621/0.7270,...,0.0451/-,0.2879/-,0.0591/-,0.3129/-,0.2633/-,0.0884/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-
7,multi_layer&i3d_flow,H3,ROIxSMC,croi_ensemble,0.1508/0.3410,0.2600/0.5398,0.2682/0.5919,0.2725/0.6043,0.2504/0.6522,0.3280/0.6768,...,0.0322/-,0.2618/-,0.0436/-,0.2841/-,0.2399/-,0.0715/-,0.2920/-,0.2087/-,0.4491/-,0.3551/-
8,single_layer&i3d_rgb+i3d_flow,H3,SMC,,0.1775/0.3777,0.2998/0.5884,0.2983/0.6173,0.2927/0.6420,0.2596/0.6649,0.3604/0.7191,...,0.0451/-,0.2888/-,0.0600/-,0.3178/-,0.2603/-,0.0892/-,0.3265/-,0.2580/-,0.4797/-,0.4134/-
9,single_layer&i3d_rgb+i3d_flow,H2,ROIxSMC,croi_ensemble,0.1826/0.3782,0.3350/0.6629,0.3257/0.6832,0.3090/0.6800,0.2886/0.7191,0.3664/0.7249,...,0.0470/-,0.2992/-,0.0636/-,0.3279/-,0.2710/-,0.0937/-,0.3296/-,0.2678/-,0.4870/-,0.4201/-


In [31]:
# load from notebook 900. (cross-notebook-ref)
submission_configs_keys = ['model_sch', 'he_sch', 'roi_sch', 'cross_roi_sch']
table_configs = [
    [
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'WB', ''),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROI', ''),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'LC', ''),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'MC', ''),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'SMC', ''),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'SC', ''),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxLC', 'croi_ensemble'),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxMC', 'croi_ensemble'),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSC', 'croi_ensemble'),
    ],
    [
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROI', ''),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'SMC', ''),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_swap'),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'voxel_swap'),
    ],
    [
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxLC', 'croi_ensemble'),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxMC', 'croi_ensemble'),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSC', 'croi_ensemble'),
    ],
    [
        ('single_layer&i3d_rgb+i3d_flow', 'H1', 'ROIxSMC', 'croi_ensemble'),
        ('single_layer&i3d_rgb+i3d_flow', 'H2', 'ROIxSMC', 'croi_ensemble'),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
    ],
    [
        ('single_layer&i3d_rgb', 'H3', 'ROIxSMC', 'croi_ensemble'),
        ('single_layer&i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
        ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
        ('multi_layer&i3d_rgb', 'H3', 'ROIxSMC', 'croi_ensemble'),
        ('multi_layer&i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
        ('multi_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
        ('single_layer+multi_layer&i3d_rgb', 'H3', 'ROIxSMC', 'croi_ensemble'),
        ('single_layer+multi_layer&i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
        ('single_layer+multi_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
    ]
]
# table_configs = torch.load('./tmp/table_configs.pt')

In [32]:
from src.utils.misc import my_query_df
from pprint import pprint

for i, table_config in enumerate(table_configs):
    dfs = []
    for config in table_config:
        config_dict = {k: v for k, v in zip(submission_configs_keys, config)}
        df = my_query_df(full_table_df, equal_dict=config_dict)
        dfs.append(df)
    table_df = pd.concat(dfs)
    display(table_df)
    table_df.to_csv(f'./tables/Table {i + 1:02d}.csv')

,model_sch,he_sch,roi_sch,cross_roi_sch,WB,V1,V2,V3,V4,LOC,...,SMC2,MC2,MC1,SC3,SC4,LC1,LC2,LC3,LC4,LC5
3,single_layer&i3d_rgb+i3d_flow,H3,WB,,0.1602/0.3567,0.2302/0.4921,0.2409/0.5266,0.2510/0.5729,0.2329/0.6239,0.3437/0.6975,...,0.0371/-,0.2670/-,0.0488/-,0.2886/-,0.2458/-,0.0766/-,0.3118/-,0.2401/-,0.4527/-,0.3264/-
2,single_layer&i3d_rgb+i3d_flow,H3,ROI,,0.1700/0.3658,0.3333/0.6613,0.3228/0.6826,0.3061/0.6760,0.2829/0.7227,0.3582/0.7247,...,0.0417/-,0.2818/-,0.0543/-,0.3059/-,0.2581/-,0.0833/-,0.3202/-,0.2499/-,0.4723/-,0.3779/-
18,single_layer&i3d_rgb+i3d_flow,H3,LC,,0.1754/0.3710,0.2963/0.5808,0.2943/0.6075,0.2885/0.6304,0.2568/0.6552,0.3590/0.7156,...,0.0442/-,0.2788/-,0.0577/-,0.3030/-,0.2549/-,0.0859/-,0.3265/-,0.2580/-,0.4797/-,0.4134/-
20,single_layer&i3d_rgb+i3d_flow,H3,MC,,0.1753/0.3747,0.2982/0.5910,0.2965/0.6167,0.2923/0.6408,0.2593/0.6642,0.3606/0.7186,...,0.0430/-,0.2888/-,0.0560/-,0.3178/-,0.2603/-,0.0857/-,0.3265/-,0.2580/-,0.4797/-,0.4134/-
8,single_layer&i3d_rgb+i3d_flow,H3,SMC,,0.1775/0.3777,0.2998/0.5884,0.2983/0.6173,0.2927/0.6420,0.2596/0.6649,0.3604/0.7191,...,0.0451/-,0.2888/-,0.0600/-,0.3178/-,0.2603/-,0.0892/-,0.3265/-,0.2580/-,0.4797/-,0.4134/-
19,single_layer&i3d_rgb+i3d_flow,H3,SC,,0.1778/0.3778,0.3010/0.5901,0.3003/0.6199,0.2938/0.6420,0.2608/0.6643,0.3608/0.7190,...,0.0451/-,0.2935/-,0.0600/-,0.3258/-,0.2616/-,0.0898/-,0.3265/-,0.2580/-,0.4797/-,0.4134/-
6,single_layer&i3d_rgb+i3d_flow,H3,ROIxLC,croi_ensemble,0.1778/0.3766,0.3334/0.6615,0.3230/0.6823,0.3079/0.6794,0.2834/0.7249,0.3621/0.7270,...,0.0451/-,0.2879/-,0.0591/-,0.3129/-,0.2633/-,0.0884/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-
15,single_layer&i3d_rgb+i3d_flow,H3,ROIxMC,croi_ensemble,0.1775/0.3793,0.3335/0.6619,0.3231/0.6820,0.3078/0.6792,0.2837/0.7241,0.3627/0.7275,...,0.0441/-,0.2952/-,0.0576/-,0.3237/-,0.2672/-,0.0879/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-
14,single_layer&i3d_rgb+i3d_flow,H3,ROIxSMC,croi_ensemble,0.1798/0.3824,0.3339/0.6620,0.3232/0.6817,0.3078/0.6794,0.2838/0.7239,0.3627/0.7275,...,0.0462/-,0.2953/-,0.0616/-,0.3238/-,0.2672/-,0.0915/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-
1,single_layer&i3d_rgb+i3d_flow,H3,ROIxSC,croi_ensemble,0.1800/0.3823,0.3338/0.6621,0.3232/0.6818,0.3080/0.6792,0.2841/0.7226,0.3631/0.7272,...,0.0462/-,0.2984/-,0.0616/-,0.3302/-,0.2670/-,0.0919/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-


,model_sch,he_sch,roi_sch,cross_roi_sch,WB,V1,V2,V3,V4,LOC,...,SMC2,MC2,MC1,SC3,SC4,LC1,LC2,LC3,LC4,LC5
2,single_layer&i3d_rgb+i3d_flow,H3,ROI,,0.1700/0.3658,0.3333/0.6613,0.3228/0.6826,0.3061/0.6760,0.2829/0.7227,0.3582/0.7247,...,0.0417/-,0.2818/-,0.0543/-,0.3059/-,0.2581/-,0.0833/-,0.3202/-,0.2499/-,0.4723/-,0.3779/-
8,single_layer&i3d_rgb+i3d_flow,H3,SMC,,0.1775/0.3777,0.2998/0.5884,0.2983/0.6173,0.2927/0.6420,0.2596/0.6649,0.3604/0.7191,...,0.0451/-,0.2888/-,0.0600/-,0.3178/-,0.2603/-,0.0892/-,0.3265/-,0.2580/-,0.4797/-,0.4134/-
14,single_layer&i3d_rgb+i3d_flow,H3,ROIxSMC,croi_ensemble,0.1798/0.3824,0.3339/0.6620,0.3232/0.6817,0.3078/0.6794,0.2838/0.7239,0.3627/0.7275,...,0.0462/-,0.2953/-,0.0616/-,0.3238/-,0.2672/-,0.0915/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-
17,single_layer&i3d_rgb+i3d_flow,H3,ROIxSMC,croi_swap,0.1795/0.3821,0.3333/0.6613,0.3228/0.6826,0.3066/0.6783,0.2829/0.7227,0.3622/0.7258,...,0.0461/-,0.2942/-,0.0614/-,0.3230/-,0.2658/-,0.0912/-,0.3276/-,0.2607/-,0.4817/-,0.4178/-
10,single_layer&i3d_rgb+i3d_flow,H3,ROIxSMC,voxel_swap,0.1907/0.3780,0.3439/0.6539,0.3322/0.6724,0.3174/0.6708,0.2912/0.7112,0.3693/0.7255,...,0.0626/-,0.3000/-,0.0773/-,0.3280/-,0.2724/-,0.1057/-,0.3328/-,0.2678/-,0.4853/-,0.4228/-


,model_sch,he_sch,roi_sch,cross_roi_sch,WB,V1,V2,V3,V4,LOC,...,SMC2,MC2,MC1,SC3,SC4,LC1,LC2,LC3,LC4,LC5
6,single_layer&i3d_rgb+i3d_flow,H3,ROIxLC,croi_ensemble,0.1778/0.3766,0.3334/0.6615,0.3230/0.6823,0.3079/0.6794,0.2834/0.7249,0.3621/0.7270,...,0.0451/-,0.2879/-,0.0591/-,0.3129/-,0.2633/-,0.0884/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-
15,single_layer&i3d_rgb+i3d_flow,H3,ROIxMC,croi_ensemble,0.1775/0.3793,0.3335/0.6619,0.3231/0.6820,0.3078/0.6792,0.2837/0.7241,0.3627/0.7275,...,0.0441/-,0.2952/-,0.0576/-,0.3237/-,0.2672/-,0.0879/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-
14,single_layer&i3d_rgb+i3d_flow,H3,ROIxSMC,croi_ensemble,0.1798/0.3824,0.3339/0.6620,0.3232/0.6817,0.3078/0.6794,0.2838/0.7239,0.3627/0.7275,...,0.0462/-,0.2953/-,0.0616/-,0.3238/-,0.2672/-,0.0915/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-
1,single_layer&i3d_rgb+i3d_flow,H3,ROIxSC,croi_ensemble,0.1800/0.3823,0.3338/0.6621,0.3232/0.6818,0.3080/0.6792,0.2841/0.7226,0.3631/0.7272,...,0.0462/-,0.2984/-,0.0616/-,0.3302/-,0.2670/-,0.0919/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-


,model_sch,he_sch,roi_sch,cross_roi_sch,WB,V1,V2,V3,V4,LOC,...,SMC2,MC2,MC1,SC3,SC4,LC1,LC2,LC3,LC4,LC5
5,single_layer&i3d_rgb+i3d_flow,H1,ROIxSMC,croi_ensemble,0.1829/0.3755,0.3354/0.6587,0.3248/0.6797,0.3109/0.6704,0.2877/0.7115,0.3662/0.7220,...,0.0473/-,0.3003/-,0.0643/-,0.3310/-,0.2701/-,0.0944/-,0.3291/-,0.2672/-,0.4874/-,0.4208/-
9,single_layer&i3d_rgb+i3d_flow,H2,ROIxSMC,croi_ensemble,0.1826/0.3782,0.3350/0.6629,0.3257/0.6832,0.3090/0.6800,0.2886/0.7191,0.3664/0.7249,...,0.0470/-,0.2992/-,0.0636/-,0.3279/-,0.2710/-,0.0937/-,0.3296/-,0.2678/-,0.4870/-,0.4201/-
14,single_layer&i3d_rgb+i3d_flow,H3,ROIxSMC,croi_ensemble,0.1798/0.3824,0.3339/0.6620,0.3232/0.6817,0.3078/0.6794,0.2838/0.7239,0.3627/0.7275,...,0.0462/-,0.2953/-,0.0616/-,0.3238/-,0.2672/-,0.0915/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-


,model_sch,he_sch,roi_sch,cross_roi_sch,WB,V1,V2,V3,V4,LOC,...,SMC2,MC2,MC1,SC3,SC4,LC1,LC2,LC3,LC4,LC5
16,single_layer&i3d_rgb,H3,ROIxSMC,croi_ensemble,0.1777/0.3779,0.3338/0.6605,0.3224/0.6795,0.3053/0.6753,0.2808/0.7183,0.3594/0.7210,...,0.0459/-,0.2915/-,0.0610/-,0.3198/-,0.2637/-,0.0904/-,0.3218/-,0.2597/-,0.4768/-,0.4161/-
4,single_layer&i3d_flow,H3,ROIxSMC,croi_ensemble,0.1565/0.3372,0.2551/0.5321,0.2619/0.5802,0.2692/0.6023,0.2530/0.6422,0.3322/0.6609,...,0.0342/-,0.2665/-,0.0476/-,0.2877/-,0.2456/-,0.0755/-,0.3022/-,0.2200/-,0.4543/-,0.3501/-
14,single_layer&i3d_rgb+i3d_flow,H3,ROIxSMC,croi_ensemble,0.1798/0.3824,0.3339/0.6620,0.3232/0.6817,0.3078/0.6794,0.2838/0.7239,0.3627/0.7275,...,0.0462/-,0.2953/-,0.0616/-,0.3238/-,0.2672/-,0.0915/-,0.3278/-,0.2608/-,0.4822/-,0.4186/-
0,multi_layer&i3d_rgb,H3,ROIxSMC,croi_ensemble,0.1747/0.3726,0.3335/0.6768,0.3221/0.6889,0.3059/0.6795,0.2812/0.7183,0.3546/0.7179,...,0.0444/-,0.2873/-,0.0584/-,0.3153/-,0.2596/-,0.0876/-,0.3216/-,0.2514/-,0.4699/-,0.4191/-
7,multi_layer&i3d_flow,H3,ROIxSMC,croi_ensemble,0.1508/0.3410,0.2600/0.5398,0.2682/0.5919,0.2725/0.6043,0.2504/0.6522,0.3280/0.6768,...,0.0322/-,0.2618/-,0.0436/-,0.2841/-,0.2399/-,0.0715/-,0.2920/-,0.2087/-,0.4491/-,0.3551/-
21,multi_layer&i3d_rgb+i3d_flow,H3,ROIxSMC,croi_ensemble,0.1764/0.3771,0.3343/0.6792,0.3234/0.6941,0.3091/0.6861,0.2845/0.7268,0.3583/0.7271,...,0.0446/-,0.2904/-,0.0587/-,0.3191/-,0.2621/-,0.0883/-,0.3255/-,0.2535/-,0.4772/-,0.4211/-
13,single_layer+multi_layer&i3d_rgb,H3,ROIxSMC,croi_ensemble,0.1788/0.3781,0.3375/0.6786,0.3256/0.6929,0.3087/0.6834,0.2840/0.7217,0.3597/0.7161,...,0.0464/-,0.2930/-,0.0615/-,0.3219/-,0.2645/-,0.0911/-,0.3231/-,0.2618/-,0.4774/-,0.4217/-
12,single_layer+multi_layer&i3d_flow,H3,ROIxSMC,croi_ensemble,0.1577/0.3387,0.2635/0.5446,0.2697/0.5942,0.2746/0.6106,0.2547/0.6489,0.3343/0.6682,...,0.0343/-,0.2669/-,0.0479/-,0.2891/-,0.2451/-,0.0759/-,0.3050/-,0.2213/-,0.4549/-,0.3573/-
11,single_layer+multi_layer&i3d_rgb+i3d_flow,H3,ROIxSMC,croi_ensemble,0.1809/0.3824,0.3383/0.6802,0.3263/0.6945,0.3107/0.6885,0.2871/0.7287,0.3630/0.7253,...,0.0468/-,0.2947/-,0.0625/-,0.3227/-,0.2671/-,0.0921/-,0.3285/-,0.2636/-,0.4830/-,0.4227/-


In [55]:
# drafts

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for model_sch, he_sch in list(itertools.product(model_schs, he_schs)):
    idf = my_query_df(score_df, {'model_sch': model_sch, 'he_sch': he_sch})
    x = idf.trained_roi_sch.unique()
    y = idf.roi.unique()
    shape = (len(x), len(y))
    scores = idf.score.values.reshape(shape)
    ax, fig = plt.subplots(figsize=(20, 4))
    sns.heatmap(scores, xticklabels=y, yticklabels=x, annot=True, fmt='.4f', cmap='Reds')
    plt.yticks(rotation=0)
    plt.title(f'model_sch={model_sch}\n he_sch={he_sch}')
    plt.show()